In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/html2023-spring-final-project/sample_submission.csv
/kaggle/input/html2023-spring-final-project/train.csv
/kaggle/input/html2023-spring-final-project/test.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd
fillvalue = {}
data = pd.read_csv("/kaggle/input/html2023-spring-final-project/train.csv")
keys = ['Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Views', 'Likes', 'Stream', 'Album_type', 'Licensed','Comments']

famous_keys = ['Views', 'Likes', 'Stream', 'Comments']
for key in keys:
    if key == 'Album_type' or key == 'Licensed':
        continue
    if key in famous_keys:
        data[key].fillna(value=0,inplace=True)
    else:
        fillvalue[key] = data[key].mean()
        data[key].fillna(value=fillvalue[key],inplace=True)
y = np.array(data['Danceability'])
new_data = {}
for key in keys:
    if key != 'Danceability':
        new_data[key] = data[key]
data = pd.DataFrame(new_data)
data

,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Stream,Album_type,Licensed,Comments
0,0.000273,3.000000,-25.17600,0.044300,7.241508e-01,0.000062,0.000807,0.340000,83.398,117907.0,0.0,137622.0,19981370.0,album,False,3070.0
1,0.184220,7.000000,-7.56137,0.095462,6.722214e-01,0.910000,0.034966,0.746000,148.692,173427.0,184825.0,5190.0,0.0,NaN,False,122.0
2,0.335839,6.000000,-15.59600,0.095462,6.676276e-01,0.867000,0.001772,0.380000,84.899,150667.0,184825.0,5190.0,19320810.0,NaN,False,122.0
3,0.209585,5.231604,-6.25100,0.027700,3.796416e-03,0.054934,0.001000,0.525842,150.220,265000.0,202457327.0,997035.0,399661898.0,album,True,24350.0
4,0.335839,5.000000,-7.56137,0.026000,3.048625e-06,0.000467,0.002924,0.295000,97.997,263867.0,316688049.0,1299086.0,0.0,album,True,31150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17165,0.794023,5.000000,-6.34400,0.032800,8.991539e-02,0.000000,0.000591,0.658000,90.002,94667.0,0.0,1113.0,0.0,NaN,True,0.0
17166,0.820026,4.000000,-1.78600,0.095462,2.195200e-05,0.000000,0.000786,0.657000,174.869,150857.0,164741.0,2019.0,10898176.0,single,True,0.0
17167,0.571787,4.000000,-4.67900,0.064700,1.148404e-01,0.000000,0.003652,0.419000,168.388,136842.0,35646.0,329.0,0.0,single,True,0.0
17168,0.451218,9.000000,-7.56137,0.419000,4.511802e-02,0.054934,0.001260,0.539000,155.378,108387.0,6533.0,0.0,6873961.0,single,True,0.0


In [3]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import normalize
from sklearn import preprocessing
X_temp_scaler = None
X_famous_scaler = None
def datatoX(data,scale=False):
    global X_temp_scaler
    global X_famous_scaler
    famous_keys = ['Views', 'Likes', 'Stream', 'Comments']
    famous_X = []
    temp_X = []
    type_X = []
    for key in data:
        if key == 'Album_type':
            types = data['Album_type'].unique()
            for t in types:
                type_X.append(np.array([1 if ty==t else 0 for ty in data['Album_type']]))
        elif key == 'Licensed':
            type_X.append(np.array([1 if l else 0 for l in data['Licensed']]))
            type_X.append(np.array([0 if l else 1 for l in data['Licensed']]))
        elif key in famous_keys:
            famous_X.append(np.array([np.log(max(a,1)) for a in data[key]]))       
        elif key in keys:
            temp_X.append(np.array([a for a in data[key]]))
#             temp_X.append(np.array([1/(a+0.000001) for a in data[key]]))
    temp_X = np.array(temp_X)
    famous_X = np.array(famous_X)
    if scale:
        X_temp_scaler = preprocessing.StandardScaler().fit(temp_X.T)
        X_famous_scaler = preprocessing.StandardScaler().fit(famous_X.T)
    temp_X = X_temp_scaler.transform(temp_X.T)
    famous_X = X_famous_scaler.transform(famous_X.T)
    type_X = np.array(type_X).T
    poly = PolynomialFeatures(3)
    temp_X = poly.fit_transform(temp_X)
    X = np.concatenate((temp_X,famous_X,type_X),axis=1)
    print(X.shape)
    return X
    
X = datatoX(data,scale = True)
X

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(17170, 296)


array([[ 1.        , -1.49878676, -0.70084108, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.        , -0.67719886,  0.55536967, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.        ,  0.        ,  0.24131698, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.        ,  1.05384844, -0.38678839, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  0.51533193,  1.18347504, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  2.18612248,  0.24131698, ...,  0.        ,
         1.        ,  0.        ]])

In [4]:
test_data = pd.read_csv("/kaggle/input/html2023-spring-final-project/test.csv")
test_data.describe()

,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Stream,id,Comments
count,5.347000e+03,5394.000000,5361.000000,5307.000000,5.369000e+03,5393.000000,5.327000e+03,5366.000000,5354.000000,5395.000000,5.388000e+03,5.371000e+03,5.360000e+03,6315.000000,5.373000e+03
mean,2.737771e-01,4.839637,-8.434696,0.100862,1.531935e-01,0.114404,2.776079e-02,0.488375,114.903993,231341.409222,8.632007e+07,5.836651e+05,1.342002e+08,20327.000000,2.838440e+04
std,2.453962e-01,3.356452,4.726596,0.094452,2.389201e-01,0.226652,8.299796e-02,0.240407,28.748170,104951.691104,2.767327e+08,1.694522e+06,2.284756e+08,1823.127807,1.213519e+05
min,8.398263e-10,0.000000,-41.530000,0.000000,2.064251e-16,0.000000,2.558369e-14,0.000000,0.000000,-391561.349824,-1.223997e+09,-6.360385e+06,-7.133725e+08,17170.000000,0.000000e+00
25%,6.670099e-02,2.000000,-10.464442,0.038600,1.225043e-03,0.000212,8.553787e-04,0.296000,93.999500,184291.608494,1.647188e+06,1.053000e+04,1.784380e+07,18748.500000,6.610000e+02
50%,2.169998e-01,5.000000,-7.385607,0.062429,2.641275e-02,0.002442,2.689712e-03,0.489771,110.298491,230387.456170,1.090642e+07,8.086501e+04,6.019122e+07,20327.000000,4.357000e+03
75%,4.152809e-01,8.000000,-5.247000,0.134240,2.048771e-01,0.095548,1.950949e-02,0.671975,133.177608,272632.988341,9.208968e+07,6.261509e+05,1.721442e+08,21905.500000,1.851600e+04
max,9.978081e-01,10.000000,3.285857,0.961000,9.999658e-01,0.982000,9.602613e-01,0.998563,209.695000,903928.210970,8.079649e+09,5.078865e+07,2.110574e+09,23484.000000,4.252791e+06


In [5]:
def preprocess(data):
    keys = ['Energy', 'Key', 'Loudness', 'Speechiness',
           'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
           'Duration_ms', 'Views', 'Likes', 'Stream', 'Album_type', 'Licensed','Comments']

    famous_keys = ['Views', 'Likes', 'Stream', 'Comments']
    for key in keys:
        if key == 'Album_type' or key == 'Licensed':
            continue
        if key in famous_keys:
            data[key].fillna(value=0,inplace=True)
        else:
            data[key].fillna(value=fillvalue[key],inplace=True)
    for key in keys:
            new_data[key] = data[key]
    return pd.DataFrame(new_data)
test_data = preprocess(test_data)
test_data

,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Stream,Album_type,Licensed,Comments
0,0.350403,6.000000,-6.679000,0.177000,5.842771e-07,0.002330,0.230346,0.525842,138.559000,222640.000000,6.935552e+08,6.220896e+06,1.040235e+09,album,True,169907.0
1,0.347429,8.000000,-5.815000,0.030200,1.148404e-01,0.000687,0.000099,0.852000,92.761000,200173.000000,7.201164e+07,0.000000e+00,3.100837e+08,NaN,True,31003.0
2,0.786330,1.000000,-3.930000,0.052200,7.676563e-05,0.054934,0.001561,0.551000,108.014000,215150.000000,8.435055e+06,2.821420e+05,6.306347e+07,single,True,7399.0
3,0.403583,2.000000,-5.810000,0.026000,3.442951e-15,0.509000,0.000262,0.578000,120.423000,233867.000000,0.000000e+00,1.788577e+06,4.346636e+08,album,True,55229.0
4,0.334255,5.231604,-8.627000,0.171000,1.619428e-05,0.000000,0.000340,0.525000,167.953000,340920.000000,6.184810e+08,6.197318e+06,6.172597e+08,NaN,True,155930.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,0.160377,7.000000,-6.366894,0.045451,3.176921e-01,0.006016,0.001642,0.160253,72.136829,233721.956648,9.636781e+06,5.807853e+04,1.750041e+07,album,True,168.0
6311,0.318585,10.000000,-4.148566,0.073000,7.419596e-03,0.000817,0.008843,0.614366,120.937819,138726.388393,4.572846e+07,7.227992e+04,1.365601e+07,single,False,448.0
6312,0.426223,5.000000,-5.807545,0.072906,6.617082e-01,0.898308,0.000662,0.087281,87.969239,185842.047750,-3.451740e+06,2.305163e+04,1.849094e+07,single,True,0.0
6313,0.204440,5.231604,-11.563964,0.037676,3.630868e-03,0.491785,0.030605,0.482987,120.937819,180639.352865,1.728134e+07,0.000000e+00,1.199593e+08,album,True,15139.0


In [6]:
X_test_data = datatoX(test_data)
X_test_data

(6315, 296)


array([[ 1.00000000e+00,  6.50469365e-02,  2.41316981e-01, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  5.17650729e-02,  8.69422355e-01, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  2.01209540e+00, -1.32894645e+00, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.00000000e+00,  4.03694126e-01, -7.27357061e-02, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00, -5.86886212e-01,  2.78934819e-16, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00, -1.43444367e+00,  2.78934819e-16, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00]])

In [7]:
def write_prediction(predictions):
    sample_df = pd.read_csv('/kaggle/input/html2023-spring-final-project/sample_submission.csv')
    sample_df['Danceability'] = predictions
    sample_df.to_csv('submission.csv',index=False) 

In [8]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.metrics import mean_absolute_error
models = []
for c in [0.02,0.05,0.1]:
    print("training...")
    regressor = SVR(kernel = 'poly', degree=3, gamma=c, C=50000, coef0=0, tol=1e-4,cache_size=10000,verbose=True)
    regressor.fit(X_train, y_train)
    print("finished training...")
    models.append(regressor)
    predictions = regressor.predict(X_test)
    predictions = np.rint(predictions)
    predictions[predictions<0] = 0
    predictions[predictions>9] = 9
    err = mean_absolute_error(y_test,predictions)
    print(f"{c}: {err}")

training...
[LibSVM].......................................................................................................................................................................